The following is from [this article](https://medium.com/daily-programming-tips/getting-started-with-python-and-influxdb-3e24e3f6b585) in Medium and [Getting Started Instruction](https://docs.influxdata.com/influxdb/v2/install/) for InfluxDB.

In [1]:
import csv
import os
from datetime import datetime

from dotenv import load_dotenv, main
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import ASYNCHRONOUS, SYNCHRONOUS

# 0. Class for InfluxClient

In [2]:
class InfluxClient:
    def __init__(self, token, org, bucket):
        self._org = org
        self._bucket = bucket
        self._client = InfluxDBClient(url="http://localhost:8086", token=token)

    def write_data(self, data, write_option=SYNCHRONOUS):
        write_api = self._client.write_api(write_option)
        write_api.write(self._bucket, self._org, data, write_precision="s")

    def query_data(self, query):
        query_api = self._client.query_api()
        result = query_api.query(org=self._org, query=query)
        results = []
        for table in result:
            for record in table.records:
                results.append((record.get_value(), record.get_field()))
        print(results)
        return results

    def delete_data(self, measurement):
        delete_api = self._client.delete_api()
        start = "1970-01-01T00:00:00Z"
        stop = "2021-10-30T00:00:00Z"
        delete_api.delete(
            start,
            stop,
            f'_measurement="{measurement}"',
            bucket=self._bucket,
            org=self._org,
        )

In [3]:
# Credentials

load_dotenv()
token = os.getenv("TOKEN")
org = os.getenv("ORG")
bucket = os.getenv("BUCKET")

# 1. Data Preparation

In [4]:
import yfinance as yf

data = yf.download("MSFT", start="2021-01-01", end="2021-10-30")

[*********************100%%**********************]  1 of 1 completed


In [5]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,222.529999,223.000000,214.809998,217.690002,211.605301,37130100
2021-01-05,217.259995,218.520004,215.699997,217.899994,211.809418,23823000
2021-01-06,212.169998,216.490005,211.940002,212.250000,206.317352,35930700
2021-01-07,214.039993,219.339996,213.710007,218.289993,212.188507,27694500
2021-01-08,218.679993,220.580002,217.029999,219.619995,213.481384,22956200
...,...,...,...,...,...,...
2021-10-25,309.359985,309.399994,306.459991,308.130005,301.476501,17554500
2021-10-26,311.000000,312.399994,308.600006,310.109985,303.413757,28107300
2021-10-27,316.000000,326.100006,316.000000,323.170013,316.191711,52588700


In [6]:
data.to_csv("data/MSFT.csv")

# 2. Write Data

InfluxDBClient has a method called `write_api` which is used to write data into your database. Below is the code snippet for for this method:

InfluxDB supports Asynchronous and Synchronous Writes, and you can specify the write type as required.

The `data` parameter can be written in three different ways, as shown below:

### 2.1. Line Protocol String

Note that the string has to follow a particular format:

> There’s a space between the tagValue and the first fieldKey, and another space between the last fieldValue and timeStamp. While parsing, these spaces are used as separators; therefore, you have to format it in the manner shown above. Note also that in this case I assumed that the first field value, fieldValue1, is a string, while fieldValue2 is a number. Therefore, fieldValue1 should appear in quotes.

Note also that the timeStamp is optional. If no timestamp is provided, InfluxDB uses the system time (UTC) of its host machine. You can read more about the [Line Protocol here](https://docs.influxdata.com/influxdb/v2/reference/syntax/line-protocol/).

### 2.2. Data Point Structure

If you do not want to deal with the format in the Line Protocol String, you can use the Point() Class. This ensures that your data is properly serialized into line protocol.

### 2.3. Dictionary Style

Data Write Method 3

In this method, you’re passing two data points and setting the write option to `ASYNCHRONOUS`. This is Python-friendly, since the data is passed as a dictionary.

In [7]:
"""
Write Data for MSFT Stock
"""
MSFT_file = open("data/MSFT.csv")
csvreader = csv.reader(MSFT_file)

In [8]:
header = next(csvreader)

In [9]:
header

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [10]:
rows = []
for row in csvreader:
    date, open, high, low = row[0], row[1], row[2], row[3]
    line_protocol_string = ""
    line_protocol_string += f"MSFT_{date},"
    line_protocol_string += f"stock=MSFT "
    line_protocol_string += f"Open={open},High={high},Low={low} "
    line_protocol_string += str(int(datetime.strptime(date, "%Y-%m-%d").timestamp()))
    rows.append(line_protocol_string)

In [11]:
rows[:5]  # Sample

['MSFT_2021-01-04,stock=MSFT Open=222.52999877929688,High=223.0,Low=214.80999755859375 1609686000',
 'MSFT_2021-01-05,stock=MSFT Open=217.25999450683594,High=218.52000427246094,Low=215.6999969482422 1609772400',
 'MSFT_2021-01-06,stock=MSFT Open=212.1699981689453,High=216.49000549316406,Low=211.94000244140625 1609858800',
 'MSFT_2021-01-07,stock=MSFT Open=214.0399932861328,High=219.33999633789062,Low=213.7100067138672 1609945200',
 'MSFT_2021-01-08,stock=MSFT Open=218.67999267578125,High=220.5800018310547,Low=217.02999877929688 1610031600']

In [12]:
IC = InfluxClient(token, org, bucket)

In [13]:
IC.write_data(rows)

# 3. Reading the Data

`InfluxDBClient` also has a method called `query_api` that can be used to read data. You can use queries for various purposes, such as filtering your data based on a specific date, aggregating your data within a time range, finding the highest/lowest values in a time range, and more. They are similar to queries you would use in SQL. You’ll need to use queries when reading data from InfluxDB.

The following code is for our class’s read method:

Here, it accepts a query and then executes it. The return value of the query is a collection of Flux Objects that match your query. The Flux Object has the following methods:

Two query examples are shown below that demonstrate the `query_data` function in action. The first query returns the high value for MSFT stock since October 1, 2021, and the second query returns the high value for the MSFT stock on 2021-10-29.

In [14]:
"""
    Return the High Value for MSFT stock for since 1st October,2021
"""
query1 = 'from(bucket: "test_db")\
|> range(start: 1633124983)\
|> filter(fn: (r) => r._field == "High")\
|> filter(fn: (r) => r.stock == "MSFT")'

In [15]:
IC.query_data(query1)

[(287.75, 'High'), (290.3999938964844, 'High'), (293.6300048828125, 'High'), (296.6400146484375, 'High'), (296.6400146484375, 'High'), (297.9700012207031, 'High'), (295.44000244140625, 'High'), (297.2799987792969, 'High'), (303.2699890136719, 'High'), (304.45001220703125, 'High'), (308.2099914550781, 'High'), (309.29998779296875, 'High'), (309.70001220703125, 'High'), (311.0199890136719, 'High'), (311.0899963378906, 'High'), (309.3999938964844, 'High'), (312.3999938964844, 'High'), (326.1000061035156, 'High'), (324.8699951171875, 'High'), (332.0, 'High')]


[(287.75, 'High'),
 (290.3999938964844, 'High'),
 (293.6300048828125, 'High'),
 (296.6400146484375, 'High'),
 (296.6400146484375, 'High'),
 (297.9700012207031, 'High'),
 (295.44000244140625, 'High'),
 (297.2799987792969, 'High'),
 (303.2699890136719, 'High'),
 (304.45001220703125, 'High'),
 (308.2099914550781, 'High'),
 (309.29998779296875, 'High'),
 (309.70001220703125, 'High'),
 (311.0199890136719, 'High'),
 (311.0899963378906, 'High'),
 (309.3999938964844, 'High'),
 (312.3999938964844, 'High'),
 (326.1000061035156, 'High'),
 (324.8699951171875, 'High'),
 (332.0, 'High')]

In [16]:
'''
    Return the High Value for the MSFT stock on 2021-10-29
'''
query2 = 'from(bucket: "test_db")\
|> range(start: 1633124983)\
|> filter(fn: (r) => r._field == "High")\
|> filter(fn: (r) => r._measurement == "MSFT_2021-10-29")'

In [17]:
IC.query_data(query2)

[(332.0, 'High')]


[(332.0, 'High')]

# 4. Updating the Data

Unlike the Write and Query APIs, InfluxDB does not have an Update API.The statement below is taken from their [documentation about how they handle duplicate data points](https://docs.influxdata.com/influxdb/v2/write-data/best-practices/duplicate-points/) .

> For points that have the same measurement name, tag set, and timestamp, InfluxDB creates a union of the old and new field sets. For any matching field keys, InfluxDB uses the field value of the new point

To update a data point, you need to have their name, tag set, and timestamp and simply perform a write operation.

# 5. Deleting Data

You can delete data using `delete_api`. Below is a gist demonstrating how to delete data:

Delete functions require the measurement value of the data point. The following gist shows a simple use case of the delete function:

InfluxDB’s documentation includes a [list of best practices for writing data](https://docs.influxdata.com/influxdb/v2/write-data/best-practices/). There are also some [best practices for data layout and schema design](https://www.influxdata.com/blog/data-layout-and-schema-design-best-practices-for-influxdb/), which you should follow for the best results.

# 6. Some Practical Use Cases of Time-Series Databases

This article examined a simple use case of a TSDB to store stock values, so you could analyze historical stock prices and forecast future values. However, you could also work with IoT Devices, sales data, and any other data series which is time-varying.

Some other practical use cases include:

1. [Time series forecasting using Tensorflow and InfluxDB](https://www.influxdata.com/blog/introduction-time-series-forecasting-tensorflow-influxdb/)

2. [Integrating InfluxDB with IFTTT to monitor your smart home](https://www.influxdata.com/blog/monitoring-smart-home-influxdb-ifttt/)

3. [Monitoring your internet speed](https://www.influxdata.com/blog/how-monitor-internet-speed-telegraf-influxdb-cloud/)

# 7. Conclusion

Hopefully this guide empowered you to set up your own TSDB instance of InfluxDB. You learned how to build a simple app to perform CRUD Operations using InfluxDB’s Python client library, but if you want to take a closer look at anything, you can find the repo with the entire source code [here](https://github.com/rahulbanerjee26/InfluxDB-Tutorial/blob/main/__init__.py).

Check out [InfluxDB’s open-source TSDB](https://www.influxdata.com/time-series-database/). It’s got client libraries for ten programming languages including Python, C++, and JavaScript, and it’s also got a lot of built-in visualization tools so you can see exactly what your data is doing.